# Сверточные нейронные сети


## Полносвязная нейронная сеть 
Fully-connected Neural Network (FCN). В современных статьях чаще используется термин Multilayer Perceptron (MLP).

На прошлом занятии мы рассмотрели следующий подход для классификации изображений:

1. Превращаем исходные данные в вектор. 

***Примечание***: при обработке цветного изображения размером $32\times32$ пикселя ($32\times32\times3$), размерность входного вектора будет равна $3072$. Однако, в общем случае модель получает на вход набор одноразмерных векторов (матрицу), потому при обработке одного изображения размер матрицы будет $3072\times1$.

2. Перемножаем матрицу данных с матрицей весов. Размер последней может быть, например, $100\times3072$. Где $3072$ - размер входного вектора, а $100$ - количество признаков, которое мы хотим получить. Результат обработки одного изображения будет иметь размер $100\times1$.

3. Поэлементно применяем к полученной матрице нелинейную функцию (функцию активации), например Sigmoid или ReLu. Размерность данных при этом не меняется ($100\times1$). В результате получаем вектор активаций или признаков.

4. Используем полученные признаки как входные данные для нового слоя. Количество весов слоя будет зависеть от размерности входной матрицы и того, что мы хотим получить на выходе. Если мы делаем классификатор на $10$ классов, то матрица весов должна иметь размерность $10\times100$, и на этом можно остановиться. Но в общем случае количество слоев может быть произвольным.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_1.png" width="700">


На изображении представлена описанная выше нейронная сеть, функцией активации в которой является ReLu. Добавление второго слоя позволило модели использовать более одного шаблона на класс. Можно убедиться в этом, обучив модель на датасете CIFAR-10 и визуализировав веса первого слоя модели.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/mlp-templates.png" width="400">

Засчёт создания нескольких шаблонов для каждого из классов, многослойные архитектуры в общем случае показывают более высокую, чем перцептроны, эффективность на задачах классификации изображений. Однако подход с использованием многослойного перцептрона также имеет свои недостатки. Чтобы обнаружить их, присмотримся к происходящему с данными при предобработке.

## Нарушение связей между соседними пикселями

Единственным этапом предобработки данных при использовании многослойного перцептрона является превращение изображения в вектор. Основной его проблемой является "потеря связи" между соседними пикселями, что показано на изображении ниже. В общем случае, значения из соседних пикселей могут оказаться на большом расстоянии внутри результирующего вектора. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_9.png" width="700">

Словом, после преобразования не получает информацию о взаимном расположении значений на исходном изображении - при подборе весов никак не используется информация о том, что какие-то значения были "близки" друг к другу на исходном изображении. При этом мы, люди, понимаем важность взаимосвязей между пикселями.  

К примеру, набор тёмных пикселей находящихся поблизости внутри изображения может сообщить нам, что там располагается некий тёмный объект. Если же мы преобразуем изображение в вектор и эти значения "разбросает" по нему без сохранения пространственной структуры, то новому наблюдателю будет очень сложно понять, что где-то на исходном изображении была тёмная зона.  

Словом, передача нейросети информации о взаимном расположении пикселей может быть полезна. Чтобы понять, каким образом это можно сделать, рассмотрим как мозг обрабатывает визуальную информацию.

## Рецептивные поля

В процессе зрительного восприятия, человек обрабатывает информацию о свете, попадающем на сетчатку глаза. Фоторецепторы сетчатки (палочки и колбочки) передают информацию в более сложные клетки - биполярные. При этом, биполярные клетки получают информацию из располагающихся рядом рецепторных клеток, таким образом обрабатывая локальную информацию. На следующих уровнях обработки информации происходит аналогичная картина - клетки уровнем выше собирают информацию из нескольких более простых близко расположенных клеток. 

В результате, каждая из сложных клеток обрабатывают информацию с нескольких простых рецепторов расположенных рядом и возбуждается при определённой комбинации сигналов от них. Область сетчатки глаза, информация из которой попадает на нервную клетку, называется рецептивным полем этой клетки.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_10.png" width="700">

Затем импульсы от зрительных клеток последовательно обрабатываются различными отделами мозга - таламусом, зрительной корой и далее, после чего извлечённая информация используется человеком для принятия решений. Заметим, что при принятии решений мы можем использовать всю информацию о том, что видим, а также визуально распознавать такие сложные формы, как лица или объекты. Обе этих возможности существуют благодаря иерархической структуре обработке информации: от отдела к отделу мозгу рецептивные поля клеток увеличиваются, как и распознаваемые зрительные формы (от простейших - распознавания краёв и углов, до таких сложных, как лица).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_11.png" width="700">

##### $\color{brown}{\text{Допольнительная информация}}$

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_12.png" width="700">

###### Hubel & Wiesel,1959

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_13.png" width="700">

###### [Ice Cube Model](https://docplayer.ru/79903347-Kasyanov-evgeniy-dmitrievich-sankt-peterburg-nmic-pn-im-v-m-beh-k-m-n-fedotov-ilya-andreevich-ryazan-ryazgmu.html)

Эта гипотетическая кубическая модель придумана для пояснения устройства клеток первичной визуальной коры, а именно  – как устроены предпочитаемые ориентации и, соответственно, реакции нейронов V1. Так, V1 можно условно поделить на кубы $2 мм^3$, каждый из которых получает сигналы от обоих глаз. Клетки с одинаковыми ориентационными предпочтениями формируют горизонтальные колонки, при этом соседние вертикальные колонки имеют слегка отличающиеся ориентационные предпочтения.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/L06_14.png" width="700">

Чувствительные к цветам клетки также собраны в столбцы (также их называют каплями, гиперколонками, шариками) 0,5 мм в диаметре в зонах соответствующих превалирующих глаз (картинка с цилиндрами). Каждый такой столбец содержит реагирующие либо на красно-зеленый, либо на сине-желтый контрасты.

# Свертка с фильтром


Как реализовать идею на компьютере?

Для этого испольльзуются так называемые фильтры.

Допустим нам нужно найти на изображении некий объект (например рука).

И у нас есть изображение этого объекта.

Тогда мы можем накладывать прямоугольник с изображением интересующего нас объекта на изображение, во всех возможных положениях. В какой-то момент шаблон полностью совпадет с частью рисунка.


Как определить что произошло совпадение?

В простейшем варианте можно было бы просто посчитать попиксельную разницу, и сложить результат как это делалось в KNN.

Тогда 0- соответствовал бы максимальному отклику.

Однако на практике используют умножение. Благодаря этому, чем больше соответствует фрагмент изображения шаблону тем больше будет отклик.

Это операция называется **"свертка с фильтром"**

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-2.gif" width="300">

*В демонстрации, зеленая секция это входное изображение 5x5. Желтая матрица это фильтр, или ядро фильтра 3x3.*








In [ ]:
 # placeholder dor demo

## Примеры 'hancrafted' фильтров

Сами коэффициенты фильтра можно подбирать вручную. 
Многие известные фильтры подобранны таким образом. 

И использовать их можно не только для поиска объектов.
 

### Фильтр Гаусса

Применяется для размытия изображений, он есть в большинстве фоторедакторов.

In [ ]:
import torch
import scipy.misc
import numpy as np
from torch import nn
from skimage import data
import matplotlib.pyplot as plt

In [ ]:
cat = data.chelsea().mean(axis = 2).astype('int32')

def im_show(arr):
  plt.axis('off')
  plt.imshow(arr, cmap='gray', vmin=0, vmax=255)
  plt.show()


im_show(cat)

g_kernel5 = torch.tensor([
                     [1,4,7,4,1],
                     [4,16,26,16,4],
                     [7,26,41,26,7],
                     [4,16,26,16,4],
                     [1,4,7,4,1]]
                     ,dtype=torch.float) / 273 # sum of weights

gaussian_filter = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=g_kernel5.shape, bias=False)
gaussian_filter.weight.data = g_kernel5.unsqueeze(0).unsqueeze(0) #gaussian_kernel
gaussian_filter.weight.requires_grad = False
input_data = torch.tensor(cat).view(1,1,cat.shape[0],cat.shape[1]).type(torch.float)
out = gaussian_filter(input_data)
im_show(out[0, 0].numpy())

### Фильтр Собеля

На заре компьютерного зрения люди придумывали различные операции, которые будут находить важные элементы на изображении. К примеру, границы объекта на изображении. Примером подобной операции является фильтр Собеля. Он у Вас будет в практической работе.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/sobel_example.jpg" width="800">


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/sobel.png" width="850">

In [ ]:
# Placeholder

##### $\color{brown}{\text{Дополнительная информация}}$

###### Детектор границ Canny

На самом деле, если мы будем применять данный алгоритм без подготовки изображения и дополнительных операций, то не получим хорошее качество детекции границ. Поэтому существуют детекторы границ, которые дополняют оператор Собеля (в алгоритме ниже оператор Собеля используется на шаге 2).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/canny.png" width="850">

###### Детекторы углов

Другими примерами подобных вручную созданных детекторов признаков являются всевозможные детекторы углов. Одним из самых известных среди них является детектор углов Харриса. Мы не будем на нем останавливаться, но вы может о нем прочесть самостоятельно.

https://en.wikipedia.org/wiki/Harris_Corner_Detector


Как Вы увидите далее, сверточные нейронные сети, по факту автоматизируют процесс нахождения подобных признаков, высвобождая время исследователей и инженеров

Перемещая различные ядра свертки по изображению, можно выполнять манипуляции с изображениями. Вот как работают многие инструменты, которые можно найти в программах для редактирования изображений, таких как Photoshop или Gimp:

###### Viola Jones ...

# Сверточный слой нейросети

Мы хотим использовать фильтры в нашей модели. При хотим что бы сеть сама научилась подбирать коэффициенты ядер фильтров. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_9.png" width="700">

Вместо вектора весов размером с целое изображение используем квадратную матрицу небольшого размера.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/filter1.png" width="700">

В результате получим число, которое будет характеризовать отклик на данный фильтр. 

Операция свертки является сводится к скалярному произведению, поэтому является линейной. 

#### Цветное/многоканальное изображение

Если речь идет о цветном изображении, то входной тензор будет иметь измерение соответствующее числу цветовых каналов.

$С\times H\times W$.

Для RGB изображений число каналов равно $3$.

CIFAR-10 изображения храняться в тензорах размером $3\times32\times32$.

Тогда и ядро фильтра должно получить дополнительное измерение соответствующее числу каналов. 

Например матрица весов для фильтра $3\times3$ будет иметь размерность $3\times3\times3$.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img_license/conv1.png" width="300">

Результатом свертки входного тензора с одним фильтром фильтром будет карта активации с глубиной $1$, вне зависимости от количества каналов.

*При этом входной тензор не обязательно должен формироаться из пикселей изображения, а может быть выходом предидущего сверточнойго слоя. 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_15-1.png" width="200">


Во время прямого прохода мы смещаем фильтр по ширине и высоте входного представления и вычисляем скалярное произведение между значениями фильтра и подлежащими значениями входного представления в любой точке.

В процессе перемещения фильтра формируеется 2х мерная карта активации, которая содержит результат применения данного фильтра к каждой из областей входного представления.

В базовом случае фильтр смещается с шагов 1 но это - параметр который может меняться.




<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_15.png" width="700">

Так же можно заметить что даже при использовании единичного шага ширина $W_{out}$ и высота $H_{out}$ **карты активаций** будет отличаться от **пространственных размерностей** исходного тензора.

In [ ]:
# let's do it manually

A = np.arange(9).reshape(3,3) # our matrix
C = np.array([[-1, 0], [0, 1]]) # filter

res = [[np.sum(A[:2, :2] * C), np.sum(A[:2, 1:3] * C)],
       [np.sum(A[1:3, :2] * C), np.sum(A[1:3, 1:3] * C)]]

print("Input: \n",A)
print("Filter: \n",C)
print("Result: \n",res)

Фильтров может быть несколько.

В результате обучения каждый должен обучиться реагировать на определенный паттерн во входном тензоре

Применения каждый даст свою карту активации размерностью $1\times H_{out}\times W_{out}$

Если объединить (стековать) эти карты между собой то получим тензор размерности $K\times H_{out} \times W_{out}$


Где K - количество фильтров.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/depth.jpeg" width="400">


На изображении результат приминения к изображению из CIFAR-10 сверточного слоя содержащего 5 фильтров.

В документации Pytorch вместо K используется обозначение $С_{out}$ 

То есть результат применения фильтра трактуется аналогично цветовому каналу на самом первом слое.



Оценим количество праметров и количество операций ...

### Сверточный слой = Сверка + активация

В результате прохода по изображению мы получаем карту активации, также известную как карта признаков. Этот процесс называется пространственной свёрткой

К изображению можно применять множество фильтров и получать на выходе разные карты активации.


По тем же соображениям что и для линейного слоя, на для каждого фильтра обучается еще обин коэффициент отвечающий за смежение (bias)


При этом сама операция свертки остается линейной операцией, и к выходу сверточного слоя применяется функция активации (например ReLu).

*Так как функция активации применяется поэлементно, не важно какой именно слой находился передней ней полносвязанный или сверточный. 


In [ ]:
import torch
from torch import nn

net = torch.nn.Sequential(
                           nn.Conv2d(3,6,3),
                           nn.ReLU(), # Activation doesn't depend on input shape
                           nn.Flatten(),
                           nn.Linear(5400,100), # H = 1 + (32 - 3) / 1 
                           nn.ReLU(),  # Activation doesn't depend on input shape
                           nn.Linear(100,10)
                           )
dummy_input = torch.randn((1,3,32,32))
out = net(dummy_input)
print(out)

## Сравнение сверточного и полносвязанного слоев 

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/conv_layer.png" width="700">

- Сколько весов у такого слоя?
- А сколько операций умножения потребуется сделать при прямом проходе?

А теперь посчитаем эти же парметры для полносвязанной сети - сети?

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_1.png" width="700">


#### Память и быстродействия

**Fully-connected Neural Network**
3 слоя  
$3\times32\times32 = 3072$  
$3072\to512$  
$512\to128$  
$128\to10$  

$W = (3072 + 1) \cdot 512 + (512 + 1) \cdot 128 + (128 + 1) \cdot 10 = 1\,573\,376 + 65\,664 + 1290 = 1\,640\,330 \approx 6.26 \text{Mb}$ (10% от датасета CIFAR10!)  
$\text{FLOPS} = 3072 \cdot 512 + 512 \cdot 128 + 128 \cdot 10 = 1\,572\,864 + 65\,536 + 1280 = 1\,639\,680 \approx 1.64e6$

**Convolution Neural Network**
4 слоя

$3\times32\times32 * 10\times3\times3 \to 10\times32\times32$  
$10\times32\times32 * 128\times3\times3 \to 128\times32\times32$  
$128\times32\times32 * 512\times3\times3 \to 512\times32\times32$  
$512\times32\times32 = 524288$  
$524288\to10$

$W = (3\cdot3\cdot3 + 1) \cdot 10 + (10\cdot3\cdot3 + 1) \cdot 128 + (128\cdot3\cdot3 + 1) \cdot 512 + 524288 \cdot 10 = 280 + 11\,648 + 590\,336 + 5\,242\,880 = 5\,845\,144 \approx 22.30 \text{Mb}$ 

$\text{FLOPS} = 30\cdot30\cdot(3\cdot3\cdot(3\cdot10 + 10\cdot128 + 128\cdot512)) + 524\,288 \cdot 10 = 900 \cdot 9 \cdot (30 + 1280 + 65\,536) + 5\,242\,880 = 546\,695\,480 \approx 5.47e8$


Свертки содержат значительно меньшее количество обучаемых параметров (не похоже). Но требуют больше вычислительных ресурсов (GPU)

## Параметры сверточного слоя

Основные параметры операции свертки слоя это:

- Размер ядра фильтра ($F$)
- Шаг ($\text{stride}$)

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_18.png" width="200">

Размерность выходных данных напрямую зависит от этих параметров следующего слоя.

Для тензоров у которых высота равна ширине ($H = W = N$)

Вичислить пространственне размеры выхода можно по формуле:

$$ N_{new} = 1 + \frac{N - F}{\text{stride}}$$

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_19.png" width="250">


При некоторых комбинациях этих двух параметров часть данных может не попасть в свертку. 

$N = 7, F = 3$

$\displaystyle \text{stride }1 \implies \frac{7 - 3}{1} + 1 = 5$ - ОК  

$\displaystyle \text{stride }2 \implies \frac{7 - 3}{2} + 1 = 3$ - ОК

stride 3 => (7 - 3)/3 + 1 = **2.33**  

В данном случае часть изображения в свертку не попадет

In [ ]:
import torch

# Create torch tensor 7x7
input = torch.tensor([[[[1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99],
                     [1,1,1,1,1,1,99]]]],dtype=torch.float)
print(input)
conv = torch.nn.Conv2d(in_channels=1, # Number of channels
                       out_channels=1, # Number of filters
                       kernel_size=3, 
                       stride = 3,
                       bias = False # Don't use bias
                       )
print(conv.weight.shape) # shape = [1,1,3,3] batch, channels, H, W
conv.weight  = torch.nn.Parameter(torch.ones((1,1,3,3))) # Replace randow weights to ones
activation = conv(input) 
print(activation)

### Количество фильтров

Задача свертки научиться разпознавать паттерны которые будут часто встречаться на изображениях в нашем датасете.

Поэтому в слой свертки добавляют не один а несколько фильтров.

От количества фильтров будет зависеть количество каналов в выходном тензоре.

Поэтому в классе Pytorch он на зывается out_channels


In [ ]:
conv = torch.nn.Conv2d(in_channels=3, # Number of input channels (3 for RGB images)
                       out_channels=64, # Number of filters
                       kernel_size=3)

dummy_input = torch.randn((1,3,32,32))
out = conv(dummy_input)
print(out.shape) # [1, 64, 30, 30] batch, C_out, H_out, W_out


### Изменение размера за счет краев

Как можно видеть из примера, при каждой операции свертки, пространственые размеры выхода уменьшаются.


В результате этого пиксели, которые находились на границе изображения участвуют в меньшем количестве сверток, чем те что были ближе к центру. 



<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_20.png" width="900">

### Расширение (padding)  

Что бы избавиться от этого эффекта используется дополнение входного тензора (англ. padding). Обозначается как P.

 Выходы с предыдущего слоя дополняются так, чтобы после свертки размер выходного тензора совпадал с входным.

 Такие свертки называют одинаковыми (англ. same convolution), а свертки без дополнения изображения называются правильными (англ. valid convolution).

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_21.png" width="250">

Чтобы добиться эффекта сохранения размера, при единичном шаге (stride == 1)

padding (P) можно рассчитать по формуле 

P = (F-1)/2


*F - размер фильтра*

 Среди способов, которыми можно заполнить новые пиксели, можно выделить следующие:

zero shift: 00[ABC]00;

border extension: AA[ABC]CC;

mirror shift: BA[ABC]CB;

cyclic shift: BC[ABC]AB.

In [ ]:
print(input.shape,input)
conv = torch.nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, stride = 1,padding = int((3-1)/2),padding_mode='zeros', bias = False)
conv.weight  = torch.nn.Parameter(torch.ones((1,1,3,3))) # Replace random weights to ones one filter, one channel,  with 3x3 kernel 
activation = conv(input) 
print(activation.shape,activation)

### Список параметров

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/conv_sum.png" width="700">

## Визуализация

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-3.gif" width="700">



## Операция свертки в PyTorch

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_23.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv2_2.png" width="700">
<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/conv2_3.png" width="700">

# Применение сверточных слоев

### Количество слоев и размер фильтров

* Сколько сверточных слоев следует добавлять в сеть?
* Какой размер ядра фильтров использовать?

Если проводить аналогию я работой мозга, то фильтры на первых слоях сети должны реагировать на простые объекты занимающую малуч часть изображения. Или иметь малое рецептивное поле.


На последних же слоях фильтры должны реагировать на признаки описывающие целые объекты, возможно занимающие все большую часть исходного изображения.

Сответственно рецептивное поле фильтров на  последних слоях сети должно быть большим, сопоставимым с размером исходного изображения.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/r-field.png" width="700">

Увеличить рецептивное поле можно двумя способами:

* увеличивать размер ядра фильтра
* Увеличивать количество слоев (см. схему)


Второй случай требует большого количества слоев.
И что бы уменьшить вычислительную сложность размерности входов   внутренних слоев последовательно уменьшают.



### Уменьшение размеров слоев 

- <font color=red >Не использовать расширение (padding)</font> 
- <font color=black >Увеличить шаг</font> 
- <font color=green >Свертка с фильтром 1x1</font> 
- <font color=green >Уплотнение (Субдискретизация, Pooling)</font> 

#### Увеличение шага свертки

Можно увеличить шаг свертки. Напрмер что бу уменьшить размер в две раза можно применить свертку с ядром 2x2 и шагом 2. 
Этот подход используется довольно часто. 


Но если задача только в уменьшении количества данных, то от требует дополнительных резурсов и памяти. 

In [ ]:
import torch
conv = torch.nn.Conv2d(in_channels=3, # Number of input channels
                       out_channels=64, # Number of filters
                       kernel_size=2,
                       stride = 2)

dummy_input = torch.randn((1,3,32,32))
out = conv(dummy_input)
print("Input shape",dummy_input.shape)
print("Shape after conv with stride 2",out.shape) # [1, 64, 16, 16] batch, C_out, H_out, W_out

#### Уплотнение (Субдискретизация, Pooling)

Задача такая же как при сжатии обычых изображений: 

уменьшить объем данных, потеряв как можно меньше значимой информаци.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_24_1.png" width="700">


И способы используются аналогичные:

**Max - pooling**
Входной тензор делится на сегменты (например 2x2) в каждом канале каждого сегмента, выбирается максимальное значение.
Из этих значение составляется выходной тензор.

**Average pooling**
Аналогично за тем исключением что вместо максимального используется среднее.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_25.png" width="700">




Можно не делить изображение на сегменты, а а применять к нему pooling - фильтр, c шагом больше 1-цы.

Именно так операция рализуется в коде.



In [ ]:
import torch
input = torch.tensor([[[
                     [1,1,2,4],
                     [5,6,7,8],
                     [3,2,1,0],
                     [1,2,3,4],
                     ]]],dtype=torch.float)

max_pool = torch.nn.MaxPool2d(2, stride=2)
avg_pool = torch.nn.AvgPool2d(2, stride=2)

print("Max pooling",max_pool(input))
print("Average pooling",avg_pool(input))

Сответственно у слоя пулинга появляеются гиперпараметры:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/pool_sum.png" width="700">


В отличие от сверточного слоя слой пулинга не содержит обучаемых параметров и работает существенно быстрее.



#### Свертка с фильтром 1x1

Операция пулинга не влияет на количество каналов. А в его уплотнении тоже возникает потребность. Для этого применяют свертку размером 1x1.  

Технически это обычный сверточный слой у которого размер ядра фильтра равен 1 и шаг равен 1. А количество выходных каналов задается количеством фильтров.

Применение таких сверток позволяетпроизвольным образом менять количество каналов.


*Фактически применение свертки 1x1 эквивалентно приминению линейного слоя к каждому из каналов.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_24.png" width="700">

In [ ]:
import torch
conv = torch.nn.Conv2d(in_channels=64, # Number of input channels
                       out_channels=32, # Number of filters
                       kernel_size=1)

dummy_input = torch.randn((1,64,56,56))
out = conv(dummy_input)
print("Input shape",dummy_input.shape)
print("Shape after 1x1 conv",out.shape) # [1, 64, 56, 56] batch, C_out, H_out, W_out

##### $\color{brown}{\text{Допольнительная информация}}$

###### Другие виды сверток


###### 1D

Свертка - это математическая операция, которая объединяет две функции в третью. Например, допустим, что у нас есть две данные функции, **е(т)** а также **г(т)** и мы заинтересованы в применении одного поверх другого и вычислении площади пересечения:**f (t) * g (t) = (f * g) (t)**

В этом случае мы применяем **г(т)**(называется ядром) более **е(т)** и изменение ответа **(Е* г) (т)** согласно пересечению над областью обеих функций. Эта концепция свертки является наиболее используемой техникой обработки сигналов, поэтому она также применяется в компьютерном зрении, что можно рассматривать как обработку сигнала от нескольких датчиков RGB. Интуиция в эффективности свертки заключается в их способности фильтровать заданные входные сигналы в объединенный, более полезный результат.

В конкретном случае изображений сигнал лучше понимается с точки зрения матриц, а не форм волны. Поэтому наши функциие **(т)а** также **г(т)** теперь станетизображение *(матрица)* а также Ядро *(матрица)* соответственно. 
Еще раз, мы можем применить свертку, сдвинув одну из матриц поверх другой:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06.gif" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-1.png" width="300">

То, что вы видите выше, это просто матричное умножение скользящего окна на изображении с ядром, а затем сложение суммы. Сила сверток в контексте Computer Vision заключается в том, что они являются отличными экстракторами функций для области датчиков RGB. При индивидуальном рассмотрении каждый пиксель (датчик RGB) бессмысленно понимать, что содержит изображение. Именно отношения друг с другом в пространстве придают образу истинный смысл. Это относится к тому, как вы читаете эту статью на своем компьютере, когда пиксели, представляющие символы, и ваш мозг сопоставляют черные пиксели друг с другом в пространстве, чтобы сформировать концепцию символов.

https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html

In [ ]:
import torch
from torch import nn
#torch.nn.Conv1d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

m = nn.Conv1d(16, 33, 3, stride=2)
input = torch.randn(20, 16, 50)
output = m(input)


###### 3D

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/3d_conv.png" width="700">



In [ ]:
#torch.nn.Conv3d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')

# With square kernels and equal stride
m = nn.Conv3d(16, 33, 3, stride=2)
# non-square kernels and unequal stride and with padding
m = nn.Conv3d(16, 33, (3, 5, 2), stride=(2, 1, 1), padding=(4, 2, 0))
input = torch.randn(20, 16, 10, 50, 100)
output = m(input)
print(output.shape)

##### $\color{brown}{\text{Допольнительная информация}}$

###### Реализация

Операция свертки может быть реализованна как скалярное произведение.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm1.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm2.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm3.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm4.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm5.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cm6.png" width="700">






https://medium.com/@_init_/an-illustrated-explanation-of-performing-2d-convolutions-using-matrix-multiplications-1e8de8cd2544

###### Справочник по сверткам

https://arxiv.org/pdf/1603.07285v1.pdf

https://towardsdatascience.com/types-of-convolutions-in-deep-learning-717013397f4d


the best: 
https://towardsdatascience.com/a-comprehensive-introduction-to-different-types-of-convolutions-in-deep-learning-669281e58215


## Сверточная сеть



Теперь в ншем распоряжении появились все необходимые инструменты для создания сверточной сети.

* в начале идут несколько сверточных слоев + функция активации за каждым слоем свертки
* Затем слой пулинга
* Затем еще несколько сверток
...

Этот набор может повторяться несколько раз, до тех пор пока пространственные размеры изображения не уменьшаться достаточно сильно  

В конце добавляются один или несколько полносвязанных слоев.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/cnn.png" width="900">

#### LeNet

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/gan/lenet5.png" width="700">

Примером сети посторенной по такой архитектуре является LeNet.
Была разработана в 1989г. [Яном Ле Куном](https://en.wikipedia.org/wiki/Yann_LeCun). Сеть имела 5 слоев, из них 2 сверточных.

Применялась в США для распознавания рукописных букв на почтовых конвертах до начала 2000г.

Ниже ее реализация на Pytorch:

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 3x3 square convolution
        self.conv1 = nn.Conv2d(1, 6, 3)
        self.conv2 = nn.Conv2d(6, 16, 3)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension after pool
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x): # Expected input size 1x1x32x32
        x = self.conv1(x) # 32 -> 30 because padding = 0
        x = F.relu(x)
        x = F.max_pool2d(x, (2, 2)) # 30 -> 15  Max pooling over a (2, 2) window
        x = self.conv2(x) #   15 -> 13 ; 1  + ((N-F) / stride)
        x = F.relu(x)
        x = F.max_pool2d(x, 2) # 13->6 
        x = torch.flatten(x,1) # 1x16x6x6 -> 576   ; alternatively use nn.Flatten()
        x = F.relu(self.fc1(x)) # 576 -> 120
        x = F.relu(self.fc2(x)) # 120 -> 84
        x = self.fc3(x) # 84 -> 10
        return x


net = Net()

In [ ]:
print(net)

Давайте разберемся почему слои имеют именно такие параметры

...

Как видно данная реализация рассчитанна на одноканальные изображения размером 32x32. 

Что бы подавать на вход изображения размером например 28x28 (MNIST) необходимо поменять параметры линейных слоев.

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

In [ ]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

Код для обучения не отличается от FC сети

In [ ]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

criterion = nn.CrossEntropyLoss().cuda()

target = torch.tensor([1])  # a dummy target, for example
# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)

loss = criterion(output, target)
loss.backward() # backprop
optimizer.step()    # Does the update

# Визуализация

Что бы понять на какие визуальные шаблоны обучилась реагировать модель можно визуализировать веса фильтров и карты ативаций .



<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-5.png" width="700">

### Визуализация весов 

Веса фильтров на первом слое легко визуализировать. И результат легко интерпретируется, так как у фильтров такое же количество каналов как и у цветных изображений (3).

Ниже приведен пример того, как это можно сделать для предобученной модели на Pytorch (Alexnet)

In [ ]:
from torchvision import models,utils
import matplotlib.pyplot as plt
import numpy as np

plt.rcParams["figure.figsize"] = (20,20)

alexnet = models.alexnet(pretrained=True)
#print(alexnet)
weight_tensor = alexnet.features[0].weight.data
print(weight_tensor.shape)

img_grid = utils.make_grid(weight_tensor,pad_value=10)
plt.imshow(np.transpose(img_grid , (1, 2, 0)))


Примеры визуализации весов фильтров первого слоя для нескольких популяьных моделей обученных на ImageNet.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-6.png" width="700">

Видно что фильтры обучились реагировать на полосы или точки определенных цветов.

### Визуализация фильтров промежуточных слоев

Но веса фильтров на скрытых слоях так просто интерпретировать. 
Количество каналов на скрытых слоях обычно сильно больше 3-x.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-7.png" width="700">

Можно делать поканальную визуализацию, но она не слишком информативна.

### Feature extractor


Альтернативным способом понять насколько эффективно сеть обучилась выделять признаки является анализ карт активаций.

Например можно взять выход предпоследнего сверточного слоя и использовать его как вектор - признак описывающий объект.

 


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-8.png" width="700">

* По сути это проецирование данных в пространство меньшей размерности при помощи нейросети. Этот принцип используеся при решении множества задачь.

У объектов одного класса расстояние между векторами признаком должно быть маленьким. Если применить к полученным векторам алгоритм поиска ближайшего соседа, это еще один способ оценки качества работы модели.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-9.png" width="700">

Как видно из иллюстрации качество работы намного лучше чем у KNN для пикселей. При этом фон и ориентация объекта перестали мешать классификации.


*Подобный подход можно применять для кластеризации неразмеченных данных.




### Визуализация карт активаций

Построение значений активации нейронов в каждом слое сети в ответ на изображение или видео. 


Если на последних слоях сети, где рецептивные поля нейронов сравнимы с размерами изображения, то можно получить информацию о том, какие нейроны реагируют на какие типы изображений.

http://yosinski.com/media/papers/Yosinski__2015__ICML_DL__Understanding_Neural_Networks_Through_Deep_Visualization__.pdf

Построение значений активации нейронов в каждом слое сети в ответ на изображение или видео.

Если на последних слоях сети, где рецептивные поля нейронов сравнимы с размерами изображения, то можно получить информацию о том, какие нейроны реагируют на какие типы изображений.


<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-12.png" width="700">

Похоже что этот нейрон научился распознавать лица.

##### $\color{brown}{\text{Допольнительная информация}}$

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-13.png" width="700">

###### Активация слоев

Наиболее очевидный метод визуализации заключается в том, чтобы показать активации сети во время прямого прохода. Для сетей ReLU активации обычно начинает выглядеть относительно сгущенной и плотной, но по мере развития обучения активации обычно становится более редкой и локализованной. 

Единственная опасная ловушка, которую можно легко заметить с помощью этой визуализации, заключается в том, что некоторые карты активации могут быть равны нулю для многих различных входов. Это может указывать на мертвые фильтры и может быть симптомом высокой скорости обучения.

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-14.png" width="700">

###### Окклюзирующие части изображения
Предположим, что CNN относит изображение к классу "собака". Как мы можем быть уверены, что он действительно улавливает собаку на изображении, а не контекстуальные сигналы с фона или другого объекта? Один из способов исследовать, из какой части изображения исходит некоторое классификационное предсказание, заключается в построении графика вероятности интересующего класса (например, класса собак) в зависимости от положения объекта окклюдера. 

То есть мы перебираем области изображения, устанавливаем патч изображения равным нулю и смотрим на вероятность класса. Мы можем визуализировать вероятность как 2-мерную тепловую карту. Этот подход был использован в визуализации и понимании сверточных сетей Мэтью Зейлера.

https://cs231n.github.io/understanding-cnn/

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-15.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-16.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-17.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-18.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-19.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-20.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-21.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-22.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-23.png" width="700">

###### Визуализация Модели класса

Далее разберем методику визуализации моделей классов, изученную с помощью сверточных нейронных сетей. Этот метод визуализации состоит в численном генерировании изображения, которое является репрезентативным для класса CNN. (which is representative of the class in terms of the ConvNet class scoring model.) 

Пускай Sc(I) - оценка класса c, вычисленная классификационным слоем CNN для изображения I. Мы хотели бы найти регуляризованное изображение L2, таким, чтобы показатель Sc высоким:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/1-1.png" width="500">

где λ - параметр регуляризации. Локально-оптимальный показатель I может быть найден методом обратного распространения. Этот процесс напрямую связан с обучением CNN, где обратное распространение используется для оптимизации весов по слоям. Разница в том, что в нашем случае оптимизация выполняется по отношению к входному изображению, а веса фиксируются к тем, которые были найдены на этапе обучения.
Мы инициализировали оптимизацию с нулевым изображением (в нашем случае CNN была обучена на данных изображения с нулевым центром), а затем добавили к результату среднее изображение обучающего набора.

Стоит также отметить, что мы использовали (ненормализованные) оценки классов Sc, а не апостериоры классов, возвращаемые слоем soft-max:

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/1-2.png" width="300">

Причина в том, что максимизация класса posterior может быть достигнута за счет минимизации баллов других классов. Таким образом, мы оптимизируем Sc для того, чтобы убедиться в том, что оптимизация концентрируется только на рассматриваемом классе Sc. 

https://arxiv.org/pdf/1312.6034.pdf

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-24.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-25.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-26.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-27.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-28.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-29.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-30.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-31.png" width="700">

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06-32.png" width="700">

###### Практика

###### Pytorch. Tensor

<img src ="http://edunet.kea.su/repo/src/L06_CNN/img/L06_28.png" width="700">

https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html